In [1]:
using Distributions, Random, Optim

In [2]:
function creat_players(clubs, players_per_club, mean = 0, var = 1, lb = 0, ub = 100)
    all_players = rand(Truncated(Normal(mean, var), lb, ub), clubs * players_per_club)
    all_players = all_players / (sum(all_players) / clubs)
    players = Dict{Int64, Float64}(zip(1:clubs * players_per_club, all_players))
    return players
end

function simulating_game(club1, club2, sims = 1000000)
    λ₁ = sum(club1)
    λ₂ = sum(club2)
    X₁ = Poisson(λ₁ / λ₂)
    X₂ = Poisson(λ₂ / λ₁)
    Y₁ = rand(X₁, sims)
    Y₂ = rand(X₂, sims)
    wins₁ = sum(Y₁ .> Y₂)
    draws = sum(Y₁ .== Y₂)
    wins₂ = sum(Y₁ .< Y₂)
    return wins₁, draws, wins₂
end

function create_games(seasons, clubs, ppc)
    results = [[[0 for i in 1:4] for j in 1:clubs * (clubs - 1)] for s in 1:seasons]
    players = creat_players(clubs, ppc)
    squads = []
    for i in 1:seasons
        line = 1
        append!(squads, [reshape(shuffle(collect(1:length(players))), (clubs, ppc))])
        actual_clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
        for i in 1:clubs
            for j in 1:ppc
                actual_clubs[i, j] = players[actual_clubs[i, j]]
            end
        end

        for j in 1:clubs
            for k in 1:clubs
                if j != k
                    Xⱼ = Poisson(sum(actual_clubs[j, :]) / sum(actual_clubs[k, :]))
                    Xₖ = Poisson(sum(actual_clubs[k, :]) / sum(actual_clubs[j, :]))
                    results[i][line][1] = j
                    results[i][line][2] = rand(Xⱼ)
                    results[i][line][3] = rand(Xₖ)
                    results[i][line][4] = k
                    line += 1
                end
            end
        end
    end
    return results, squads, players
end

# results be an array like [1, 1, 0, 3]
# [home club, score home, score away, away club]

function likelihood(results, squads, players)
    if typeof(players) != Dict{Int64, Float64}
        players = Dict{Int64, Float64}(zip(1:length(players), players))
    end
    seasons = length(results)
    loglikelihood = 0
    for i in 1:seasons
        games = length(results[i])
        actual_clubs = convert(Matrix{Float64}, (deepcopy(squads[i])))
        for j in 1:size(actual_clubs)[1]
            for k in 1:size(actual_clubs)[2]
                actual_clubs[j, k] = players[actual_clubs[j, k]]
            end
        end
        
        for j in 1:games
            clubₕ, scoreₕ, scoreₐ, clubₐ = results[i][j]
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₕ, :]) / sum(actual_clubs[clubₐ, :])),
                                    scoreₕ)
            loglikelihood -= logpdf(Poisson(sum(actual_clubs[clubₐ, :]) / sum(actual_clubs[clubₕ, :])),
                                    scoreₐ)
        end
    end
    
    return loglikelihood
end

likelihood (generic function with 1 method)

In [3]:
seasons = 10
n_clubs = 20
ppc = 11

# compilando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(results, squads, players)

# reexecutando
results, squads, players = @time create_games(seasons, n_clubs, ppc)
clubs = convert(Matrix{Float64}, (deepcopy(last(squads))))
for i in 1:n_clubs
    for j in 1:ppc
        clubs[i, j] = players[clubs[i, j]]
    end
end

@time likelihood(results, squads, players)

  1.312700 seconds (2.14 M allocations: 126.706 MiB, 11.31% gc time, 99.11% compilation time)
  0.526040 seconds (2.55 M allocations: 146.270 MiB, 9.26% gc time, 98.14% compilation time)
  0.030637 seconds (69.11 k allocations: 3.847 MiB, 56.12% gc time)
  0.009588 seconds (83.26 k allocations: 3.674 MiB)


9903.857931866645

# Teste para Otimização

In [4]:
goal(x, results, squads) = likelihood(results, squads, x)
lower = zeros(220)
upper = 20 * ones(220)
x_inicial = rand(220)
# r = optimize(x -> goal(x, results, squads), x_inicial, lower, upper, Fminbox{NelderMead}())
# res = optimize(x -> goal(x, results, squads), x_inicial, Optim.Options(iterations = 10000))
res = optimize(x -> goal(x, results, squads),
               lower,
               upper,
               x_inicial,
               Fminbox(NelderMead()),
               Optim.Options(iterations = 1000))

Optim.converged(res), Optim.minimizer(res), Optim.minimum(res)

(true, [1.1043234616984905, 0.18896043203221838, 0.46403456582991787, 0.48552930003759603, 0.9031358779356152, 0.7349665552433005, 1.2000166763877982, 0.8955477029326723, 1.42938929510078, 0.3856978994106153  …  0.052286478851458824, 0.7172184427888593, 1.076141863621114, 0.13223906387508008, 0.5975136777559393, 0.869044075124104, 0.03283121901824008, 0.2821921778517665, 0.6661014936856192, 0.7323687085561844], 9822.460470283928)

In [5]:
res

 * Status: success

 * Candidate solution
    Final objective value:     9.822460e+03

 * Found with
    Algorithm:     Fminbox with Nelder-Mead

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 2.00e+01 ≰ 1.0e-08

 * Work counters
    Seconds run:   2071  (vs limit Inf)
    Iterations:    120
    f(x) calls:    203182
    ∇f(x) calls:   1
